In [1]:
import pandas as pd

equal_copy = pd.read_json('/dccstor/myu/experiments/eli5_cmsks_ctx_3/checkpoint-16452/output/eval_pred_scores.json', lines=True)
fid = pd.read_json('/dccstor/myu/experiments/eli5_fid_beam_ctx3_0720/checkpoint-32904/output/eval_pred_scores.json', lines=True)
kg_hop2 = pd.read_json('/dccstor/myu/data/kilt_eli5_dpr/eli5-dev-kilt-dpr-kg-hop2.json', lines=True)
kg_hop1 = pd.read_json('/dccstor/myu/data/kilt_eli5_dpr/eli5-dev-kilt-dpr-kg-hop1.json', lines=True)
kg_hop3 = pd.read_json('/dccstor/myu/data/kilt_eli5_dpr/eli5-dev-kilt-dpr-kg-hop3.json', lines=True)

equal_copy.sample(5)

,id,prediction_text,max_rouge_ref,rouge-L
485,48gd6m,They are not forced to accept it. They can ig...,They can ignore it and decide by themselves. T...,18.18
393,x4ey9,They don't have to pay for the rights to use t...,"Trademark says ""I own this brand name and logo...",22.22
1431,3ju6cs,"When you're sleepy, your brain shuts down the ...",If you were really sleepy then you were probab...,26.60
178,2stzdx,"I'm not a photographer, but I was a photograph...","Long exposure is one way to do it, yes, but as...",21.43
782,2yomo8,"In the US, we don't have a minimum wage. We h...",Tipping is not really 'tipping' in the US. It ...,23.47


In [2]:
ids_to_watch = {"1kiwfx", "jv91t","3ji34o","3br75b","1ircew","3ln11h","31m33z","3h8ejt","295oms",
"6efne8","3ek3j5","88u4t9","2fn69p","5kxpj5","4nktw9","809xp5","2gm843","52etuy","4lnk7x"}

In [3]:
kg_hop1.sample(5)

,id,input,output,meta,query,kg_vocab
817,161f78,Why do game company's like Microsoft and BioWa...,[{'answer': 'It encourages people to spend mor...,NaN,"[use, directly, cash, points, transacting, com...","[ongoing, stores, described, muds, game, speci..."
1388,3e1hl2,Why is it that a fully buffered YouTube video ...,"[{'answer': 'Since 2013, youtube doesn't prelo...",NaN,"[click, buffer, buffered, ahead, ?, youtube, s...","[frame, field, -, phenomenon, process, frames,..."
690,5d6xnp,Why isn't there a conservative version of the ...,"[{'answer': 'There have been attempts, but the...",NaN,"[?, conservative, version, daily]","[solo, marketed, politicians, acclaim, winstea..."
615,7rduum,How does Nintendo's labo work?,"[{'answer': 'In conjunction with the software,...",NaN,"[labo, ?, work, nintendo]","[announced, post, gaming, concept, display, ov..."
167,1eem7n,What does it mean if you get more than one lif...,[{'answer': 'If one count is overturned they s...,NaN,"[sentence, life, mean, ?]","[prisoner, penalty, offences, reformatory, pro..."


In [4]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def union(lst1, lst2):
    return list(set(lst1) & set(lst2))

def compute_overlaps(experiment, hops):
    gold_kg_avg_overlap = 0
    gen_kg_avg_overlap = 0
    gen_gold_avg_overlap = 0
    for index, example in hops.iterrows():
        experiment_paragraph = experiment[experiment["id"] == example.id].iloc[0]
        gold_words = experiment_paragraph['max_rouge_ref'].split()
        generated_words = experiment_paragraph['prediction_text'].split()

        if len(example['kg_vocab']) == 0:
            continue

        gold_intersection = intersection(gold_words,example['kg_vocab'])
        if len(gold_intersection) > 0:
            gold_kg_avg_overlap += len(gold_intersection)/len(example['kg_vocab'])
        generated_intersection = intersection(generated_words,example['kg_vocab'])
        if len(generated_intersection) > 0:
            gen_kg_avg_overlap += len(generated_intersection)/len(example['kg_vocab'])
        gen_gold = intersection(generated_words,gold_words)
        if len(gen_gold) > 0:
            gen_gold_avg_overlap += len(gen_gold)/len(example['kg_vocab'])

        if example['id'] in ids_to_watch:
        # equal copy is higher than fid
            print(example['id'] + "\t gold \t" + 
            (str(gold_words) if gold_words != None else "") + "\n\t kg \t" +
            str(example['kg_vocab']) + "\n\t gen \t" + 
            (str(gen_gold) if  gen_gold != None else ""))
            print("gold/kg: " + str(gold_intersection) + " " + str(len(gold_intersection)/len(example['kg_vocab'])))
            print("gen/kg: " + str(generated_intersection) + " " + str(len(generated_intersection)/len(example['kg_vocab'])))
            print("gen/gold: " + str(gen_gold) + " " + str(len(gen_gold)/len(example['kg_vocab'])))

    print("gold/kg: " + str(gold_kg_avg_overlap/len(hops)))
    print("gen/kg: " + str(gen_kg_avg_overlap/len(hops)))
    print("gen/gold: " + str(gen_gold_avg_overlap/len(hops)))

In [48]:
def get_good_examples(name1, experiment1, name2, experiment2, hops):
    
    count = 0
    for _, example in hops.iterrows():
        experiment_paragraph = experiment1[experiment1["id"] == example.id].iloc[0]
        experiment2_paragraph = experiment2[experiment2["id"] == example.id].iloc[0]
        # different best paragraphs
        if experiment2_paragraph['max_rouge_ref'] != experiment_paragraph['max_rouge_ref']:
            continue
        
        gold_words = experiment_paragraph['max_rouge_ref'].split()
        generated_words = experiment_paragraph['prediction_text'].split()
        generated_words2 = experiment2_paragraph['prediction_text'].split()

        if len(example['kg_vocab']) == 0:
            continue

        gold_intersection = intersection(gold_words,example['kg_vocab'])
        generated_intersection = intersection(generated_words,example['kg_vocab'])
        gen_gold = intersection(generated_words,gold_words)
        generated_intersection2 = intersection(generated_words2,example['kg_vocab'])
        gen_gold2 = intersection(generated_words2,gold_words)
        gen_gold_kg1 = intersection(gen_gold,example['kg_vocab'])
        gen_gold_kg2 = intersection(gen_gold2,example['kg_vocab'])

        # equal copy is higher than fid
        # #len(gold_intersection)/len(example['kg_vocab']) > .1 and len(generated_intersection)/len(example['kg_vocab']) and \        
        if experiment_paragraph['rouge-L'] > experiment2_paragraph['rouge-L'] and \
        len(generated_intersection) > len(generated_intersection2) and \
        len(gen_gold_kg1) > 0:
            count += 1
            print(example['id'] + " " + str(example['input']) + "\n--\ngold: " + experiment_paragraph['max_rouge_ref'] 
            + "\n--\nkg " + str(example['kg_vocab']) + 
            "\n--\ngenerated " + name1 + ": " + experiment_paragraph['prediction_text'] +
            "\n--\ngenerated " + name2 + ": " + experiment2_paragraph['prediction_text'])
            print("--\ngold/kg: " + str(gold_intersection) + " " + str(len(gold_intersection)/len(example['kg_vocab'])))
            print("gen/kg " + name1 + ": " + str(generated_intersection) + " " + str(len(generated_intersection)/len(example['kg_vocab'])))
            print("gen/kg " + name2 + ": " + str(generated_intersection2) + " " + str(len(generated_intersection2)/len(example['kg_vocab'])))
            print("gen/gold: " + str(gen_gold) + " " + str(len(gen_gold)/len(example['kg_vocab'])))
            print("gen/gold/kg " + name1 + ": " + str(gen_gold_kg1))
            print("gen/gold/kg " + name2 + ": " + str(gen_gold_kg2))
            print("rouge " + name1 + " " + str(experiment_paragraph['rouge-L']) + " > rouge " + name2 + " " + str(experiment2_paragraph['rouge-L']))
            print("------")

    print(count)
    print(len(hops))

In [69]:
print('equal copy, hop 1')
compute_overlaps(equal_copy, kg_hop1)
print('equal copy, hop 2')
compute_overlaps(equal_copy, kg_hop2)
print('equal copy, hop 3')
compute_overlaps(equal_copy, kg_hop3)
print('fid, hop 1')
compute_overlaps(fid, kg_hop1)
print('fid, hop 2')
compute_overlaps(fid, kg_hop2)
print('fid, hop 3')
compute_overlaps(fid, kg_hop3)

equal copy, hop 1
1kiwfx	 gold 	['The', 'unrealistic', 'part', 'of', 'that', 'flick', 'is', 'not', 'the', 'trading', 'but', 'Winthorp', 'and', 'Valentine', 'being', 'able', 'just', 'to', 'waltz', 'in', 'to', 'that', 'pit', 'and', 'stand', 'wherever', 'they', 'want.', 'Spots', 'in', 'a', 'commodity', 'pit', 'are', 'protected', 'like', 'gang', 'turf.', 'They', 'just', 'go', 'in', 'and', 'stand', 'in', 'the', 'middle.', 'Also', 'if', 'they', 'deposit', 'the', 'cash', 'from', "everyone's", 'savings', 'lets', 'say', '100k', 'and', 'the', 'margin', 'per', 'contract', 'is', '5k', 'per', 'contract', 'they', 'can', 'only', 'buy', 'or', 'sell', '20', 'contracts.', 'I', "don't", 'know', 'what', 'the', 'FCOJ', 'margins', 'are,', 'but', 'if', 'they', 'trade', 'more', 'than', '20', 'the', 'profit', 'goes', 'to', 'the', 'exchange.', 'At', 'least', "that's", 'how', 'the', 'CME', 'rolls.']
	 kg 	['song', 'price', 'television', 'duke', 'buying', 'paths', 'winthorpe', 'bullet', 'holdings', 'dollar', 'phi

In [49]:
get_good_examples("equal_copy", equal_copy, "fid", fid, kg_hop3)

4pmzko How would someone who is fit be affected if they literally ate complete Junk food for one day?
gold: It's still calories in vs. calories out. 10,000 is a huge amount for one day but if you eat 2,000 the other 6 days and are doing enough exercise such that your total energy use is 23,000 calories that week you're still going to lose weight.
--
kg ['diamond', 'prevents', 'medicine', 'nutritional', 'categorically', 'future', 'family', 'pejorative', 'component', 'food', '1950s', 'amounts', 'principles', 'herbert', 'deficiency', 'basic', 'b', 'levels', 'groups', 'enzyme', 'm.', 'dating', 'restaurants', 'obese', 'unintentionally', 'malnourishment', 'vitamin', 'conscious', 'calories', 'people', 'healthy', 'exercise', 'term', 'mass', 'shelton', 'described', 'adequate', 'united', 'states', 'foods', 'program', 'series', 'reasonable', 'biotin', 'determine', 'junk', 'rare', 'context', 'disorder', 'total', 'losing', 'fit', 'inadequate', 'exploration', 'effort', 'diet', 'based', 'weight', 'sy